In [5]:
import os
import numpy as np
from music21 import converter, instrument, note, chord

In [7]:
# Function to get all MIDI files from a directory
def get_midi_files(path):
    midi_files = [f for f in os.listdir(path) if f.endswith(".mid")]
    return [os.path.join(path, midi) for midi in midi_files]

# Function to extract notes and chords from MIDI files
def extract_notes(midi_paths):
    notes = []
    for midi_path in midi_paths:
        midi = converter.parse(midi_path)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        if parts:  
            notes_to_parse = parts.parts[0].recurse()  
        else:
            notes_to_parse = midi.flat.notes
        
        # Extract notes and chords
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes


In [13]:
# Get the MIDI files and extract notes
midi_folder = r"D:\lofi-music"
midi_files = get_midi_files(midi_folder)
notes = extract_notes(midi_files)
print(f"Total notes and chords extracted: {len(notes)}")

Total notes and chords extracted: 1900


In [15]:
# Prepare the mapping for notes to integers and vice versa
note_names = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(note_names)}
int_to_note = {number: note for note, number in note_to_int.items()}

# Prepare sequences of notes
sequence_length = 100  # Number of notes per sequence
network_input = []
network_output = []

In [17]:
# Create input/output sequences from notes
for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[note] for note in sequence_in])
    network_output.append(note_to_int[sequence_out])

# Reshape the input for LSTM [samples, time steps, features]
X = np.reshape(network_input, (len(network_input), sequence_length, 1))
X = X / float(len(note_names))  

# One-hot encode the output
y = np.zeros((len(network_output), len(note_names)))
for i, output in enumerate(network_output):
    y[i][output] = 1

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation

# Build the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(note_names), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')


C:\Users\asus\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
# Train the model
model.fit(X, y, epochs=200, batch_size=64)


Epoch 1/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - loss: 5.3453
Epoch 2/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - loss: 5.0546
Epoch 3/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - loss: 4.9528
Epoch 4/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - loss: 4.9291
Epoch 5/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - loss: 4.9522
Epoch 6/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: 4.9438
Epoch 7/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - loss: 4.9320
Epoch 8/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - loss: 4.9068
Epoch 9/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - loss: 4.9329
Epoch 10/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - loss: 4.8180
Epoch 11/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - loss: 4.7749
Epoch 12/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - loss: 4.7806
Epoch 13/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: 4.6852
Epoch 14/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - loss: 4.6173
Epoch 15/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - loss: 4.5577
Epoc

In [23]:
# Function to generate music using the trained model
def generate_music(model, sequence_length, note_to_int, int_to_note, seed=None, num_notes=500):
    # Generate a seed sequence from random or pre-selected notes
    if seed is None:
        start_index = np.random.randint(0, len(network_input)-1)
        prediction_input = network_input[start_index]
    else:
        prediction_input = [note_to_int[n] for n in seed]
    
    prediction_output = []
    
    # Generate notes
    for _ in range(num_notes):  # Number of notes to generate
        prediction_input_reshaped = np.reshape(prediction_input, (1, len(prediction_input), 1))
        prediction_input_reshaped = prediction_input_reshaped / float(len(note_to_int))
        
        predicted_probs = model.predict(prediction_input_reshaped, verbose=0)
        index = np.argmax(predicted_probs)
        result = int_to_note[index]
        
        prediction_output.append(result)
        
        prediction_input.append(index)  # Add predicted note to the input sequence
        prediction_input = prediction_input[1:]  # Keep the sequence length fixed
    
    return prediction_output


In [25]:
# Generate new music
generated_notes = generate_music(model, sequence_length, note_to_int, int_to_note)
print("Generated Notes:", generated_notes)

Generated Notes: ['G#4', 'E-5', 'E3', '9.11', '2.7', 'B5', 'D6', '1.2', 'A5', '6.11', 'B5', 'F#6', '2.4', 'A5', 'G5', 'B5', 'B5', 'D6', '1.2.6', 'A5', 'B5', 'B5', 'F#6', 'E6', '9.0.2.4.5', '9.0.2.4.5', 'A4', 'G4', '2.4.5.9', 'D4', '4.7.9.0', '4.7.9.0', '2.5.7.9.10', '9.0.2.5', 'C5', '8.11.3', 'F#4', 'B4', 'E5', 'E-5', 'G#4', 'F#4', 'C#4', '8.9.11.1.4', '1.4.6.8.9', '8.11.3', 'F#4', 'B4', 'E5', 'E-5', 'G#4', 'F#4', 'C#4', '8.9.11.1.4', '1.4.6.8.9', '3.6.8.11', '2.3.7.10', 'B-5', 'C6', '5.8.0', 'D6', '5.8.0', 'C6', '2.5.7.10', 'F#5', 'D5', 'G5', 'E-5', 'F5', '3', '2.3.7.10', 'B-5', 'C6', '5.8.0', 'D6', '5.8.0', 'C6', '2.5.7.10', 'F#5', 'D5', 'G5', 'E-5', 'F5', 'E-5', '3.6.8.11', 'B4', '8.11.1.4', '3.4.8.11', 'G#4', '3.6.8.11', 'B4', '8.11.1.4', '3.4.8.11', 'E4', 'F#4', 'G#4', 'B4', '0.3.5.8', '10.1.3.6', '0.1.5.8', '0.3.5.8', '10.1.3.6', '0.1.5.8', '6.7.11.2', 'F#6', 'D6', 'B5', 'E5', 'G5', '9.0.2.5', 'B5', 'A5', '11.0.4.7', 'A5', '7.10.0.3', '2.5.7.10', '4.7.9.0', '9.10.2.5', '4.7.9.0',

In [27]:
from music21 import stream, note, chord

# Convert list of notes to a MIDI file
def create_midi(notes, file_name="generated_music.mid"):
    midi_stream = stream.Stream()
    
    for pattern in notes:
        if '.' in pattern or pattern.isdigit():  # It's a chord
            chord_notes = pattern.split('.')
            chord_notes = [note.Note(int(n)) for n in chord_notes]
            midi_stream.append(chord.Chord(chord_notes))
        else:  # It's a note
            midi_stream.append(note.Note(pattern))
    
    midi_stream.write('midi', fp=file_name)

# Create a MIDI file with generated notes
create_midi(generated_notes, file_name="generated_lofi_music.mid")
print("MIDI file generated.")


MIDI file generated.


In [29]:
import os

# Print current working directory
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\asus\Documents\my files\MyProjects
